# Fire from forest

- Authors:
  - Axel Suárez Polo (@ggzor)
  - Sergio Daniel Cortez Chaves (@SerCor)

This notebook shows the training of a neural network to identify fire and smoke within sub-regions
of an image. Most of this notebook shows the process of adjusting the hyperparameters using an automatic
heuristic search (based on average precision and standard deviation).

The dataset was generated by @SerCor, using [tk-tagger](https://github.com/ggzor/tk-tagger), a 
tkinter python application to interactively tag sub-regions of an image; developed to ease the
dataset creation.

[scikit-learn](https://scikit-learn.org/stable/) is used to train the neural network, but it is used
behind a parallel multiprocessing interface with shared memory developed to speed up the training process. 
The relevant module is `experimenter.parallel`.

This notebook goes step-by-step:
  1. Configure logging and some global variables.
  2. Load the dataset using [pandas](https://pandas.pydata.org/).
  3. Configure the experiment ranges.
  4. Do hyperparameter search:
       - Optimal number of neurons and layers.
       - Optimal number of epochs
       - Optimal value for the learning rate.
       - Optimal value for the momentum.

Each step of the hyperparameters search takes the previous `BEST_FOREACH_PHASE` values (default 10) 
from the previous phase and searchs again in the specified range in the configuration.

A heatmap is shown for each experiment, showing the relation of the optimized parameter and the mean 
validation score.

Some graphics of this notebooks are not shown in the default GitHub notebook viewer, because this
notebook makes use of the [altair](https://github.com/altair-viz/altair) visualization library. The
recommended way to view this notebook is using [Visual Studio Code](https://code.visualstudio.com/) which
integrates a [vega](https://github.com/vega/vega) renderer, required by the `altair` library.


## General Configuration

In [7]:
# Enable some logging capabilities
import logging
logging.basicConfig(level='INFO')

# Disable not useful warnings
import os

os.environ["PYTHONWARNINGS"] = "ignore"

# The random seed to use for all the training initializations.
RANDOM_SEED = 0

# Set to true if you want to see a test run with an approximate duration of 5 minutes.
# Setting this value to `False` is not recommended because it can take hours.
TEST_ENVIRONMENT = False

## Data load

The data is loaded with pandas, we load all the `(train, test)` pairs from the `dataset/` folder.

In [8]:
from pathlib import Path
import pandas as pd
import re

# Get all the integer values from the dataset/ files.
filenames_integers = (
    int(m[0]) for p in Path("dataset/").iterdir() if (m := re.search("\d+", str(p)))
)

# The number of partitions to use for the cross validation
PARTITIONS = max(filenames_integers) + 1

datasets = [
    [pd.read_csv(f"dataset/{ftype}_{i}.csv") for ftype in ["train", "test"]]
    for i in range(PARTITIONS)
]

datasets[0][0]


,rgb__mean_c0,rgb__mean_c1,rgb__mean_c2,rgb__stdev_c0,rgb__stdev_c1,rgb__stdev_c2,rgb__median_c0,rgb__median_c1,rgb__median_c2,rgb__cov_0,...,hsv__median_c0,hsv__median_c1,hsv__median_c2,hsv__cov_0,hsv__cov_1,hsv__cov_2,gray__mean,gray__stdev,gray__median,tag
0,-0.536942,-0.442961,-0.672323,-0.474472,-0.430128,-0.546753,-0.500000,-0.456790,-0.687764,-0.729456,...,-0.648039,-0.096774,-0.476190,-0.144345,0.169737,-0.135806,-0.468954,-0.414427,-0.468354,OTHER
1,-0.419275,-0.081467,-0.633008,-0.668017,-0.567913,-0.634588,-0.396825,-0.135802,-0.654008,-0.844100,...,-0.508461,0.229041,-0.174603,-0.148017,0.178669,-0.059182,-0.177832,-0.586708,-0.206751,OTHER
2,-0.386719,-0.225085,-0.236443,-0.455186,-0.390740,-0.501987,-0.349206,-0.234568,-0.232068,-0.710160,...,-0.168004,-0.733333,-0.253968,-0.056420,0.146340,-0.049658,-0.245085,-0.378672,-0.248945,OTHER
3,-0.478917,-0.469440,-0.634856,-0.403600,-0.402908,-0.456290,-0.436508,-0.489712,-0.662447,-0.690361,...,-0.737659,-0.120000,-0.436508,-0.151594,0.167857,-0.217713,-0.472059,-0.368652,-0.476793,OTHER
4,-0.386173,-0.340930,-0.431716,-0.037187,-0.123209,-0.323451,-0.468254,-0.432099,-0.489451,-0.381023,...,-0.689688,-0.632116,-0.444444,-0.224676,0.067193,0.062024,-0.343443,-0.063203,-0.426160,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,0.689095,0.013688,-0.845551,-0.436299,-0.583719,-0.794527,0.722222,-0.012346,-0.864979,-0.795621,...,-0.822186,0.850307,0.722222,-0.155862,0.172441,-0.017189,0.128083,-0.566538,0.113924,SMOKE
1643,-0.464050,-0.897972,-0.984667,-0.871884,-0.951986,-0.966597,-0.452381,-0.884774,-0.991561,-0.918204,...,-0.938673,0.969697,-0.452381,-0.152948,0.172328,-0.031240,-0.803479,-0.933812,-0.789030,SMOKE
1644,0.013481,-0.643461,-0.990395,-0.809236,-0.901488,-0.951590,0.015873,-0.646091,-1.000000,-0.909680,...,-0.889935,1.000000,0.015873,-0.153965,0.171769,-0.033706,-0.511942,-0.880163,-0.518987,SMOKE
1645,0.751979,-0.016336,-0.757573,-0.982805,-0.959988,-0.984182,0.753968,-0.045267,-0.755274,-0.922824,...,-0.847000,0.743363,0.753968,-0.154411,0.170781,-0.033856,0.126442,-0.961770,0.105485,SMOKE


A label encoder is used to transform the tags of the data frames to numbers.

In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(
    [tag for experiment in datasets for df in experiment for tag in df["tag"].unique()]
)

label_encoder.classes_


array(['FIRE', 'OTHER', 'SMOKE'], dtype='<U5')

A `DataExperiment` is created for each `(train, test)` dataframe pair.

In [10]:
from experimenter.model import DataExperiment


def to_data_experiment(train_test_df) -> DataExperiment:
    train, test = train_test_df

    return DataExperiment(
        train.loc[:, train.columns != "tag"].to_numpy(),
        label_encoder.transform(train["tag"]),
        test.loc[:, train.columns != "tag"].to_numpy(),
        label_encoder.transform(test["tag"]),
    )


all_experiments = [to_data_experiment(pair) for pair in datasets]


## Experiment configuration

In [11]:
from pprint import pprint
from functools import reduce
import numpy as np

from experimenter.model import ExperimentResult

# The number of classes within the dataset
CLASSES = label_encoder.classes_.size
# The number of attributes found in the dataset
ATTRIBUTES = datasets[0][0].columns.size - 1

# A range for each optimizable hyperparameter
NEURONS_RANGE = np.arange(CLASSES, ATTRIBUTES + CLASSES + 1, 1)
LAYERS_RANGE = np.arange(1, 7 + 1, 1)
EPOCHS_RANGE = np.arange(50, 300 + 1, 10)
LEARNING_RATE_RANGE = np.around(np.linspace(0.001, 0.200, 30), 4)
MOMENTUM_RANGE = np.around(np.linspace(0.01, 0.4, 30), 4)

# Number of best parameters to keep for each training phase
BEST_FOREACH_PHASE = 15
BEST_BY_MEAN = 5
BEST_BY_CUSTOM_CRITERIA = BEST_FOREACH_PHASE - BEST_BY_MEAN


# The custom criteria to use to choose the best that go to the next phase
# It prefers low deviations and high precision
TOLERABLE_DEVIATION = 0.4
NEGATIVE_DEVIATION_WEIGHT = -10

def experiment_result_criteria(experiment: ExperimentResult) -> float:
    deviation_factor = 0
    if experiment.stddev > TOLERABLE_DEVIATION:
        deviation_factor = NEGATIVE_DEVIATION_WEIGHT * experiment.stddev

    return experiment.mean + deviation_factor


# Get the complete total number of experiments
# Note that this is just the theoretical number of experiments that had
# to be run if the best hyperparameters were to be found
total_experiments = reduce(
    lambda x, y: x * y,
    map(
        lambda arr: np.size(arr, 0),
        [
            NEURONS_RANGE,
            LAYERS_RANGE,
            EPOCHS_RANGE,
            LEARNING_RATE_RANGE,
            MOMENTUM_RANGE,
        ],
    ),
)
print(f"Complete total: {total_experiments}")


Complete total: 6552000


## Visual configuration

This variables are used to show the results in the heatmaps within using too much
vertical space.

In [12]:
NEURON_BIN_SIZE = 20
EPOCH_BIN_SIZE = 600
LEARNING_RATE_BIN_SIZE = 0.300
MOMENTUM_BIN_SIZE = 0.500

# Sometimes the accuracy is lower than this number, that makes some
# details disappear, so this treshold is used to show more details.
DETAILED_VIEW_TRESHOLD = 85

## Experimentation

A `ParallelExperimenter` is created with the `DataExperiment`'s that were created in the previous step.
This class will allow us to run our training in parallel.

In [13]:
from experimenter.ray_runner import RayExperimenter
experimenter = RayExperimenter(all_experiments, RANDOM_SEED)

## Test parameters

If the `TEST_ENVIRONMENT` flag is set, just a fraction of the hyperparameter tuning phases will be ran.

In [14]:
if TEST_ENVIRONMENT:
    NEURONS_RANGE = NEURONS_RANGE[: NEURONS_RANGE.size // 4]
    LAYERS_RANGE = LAYERS_RANGE[: NEURONS_RANGE.size // 4]
    EPOCHS_RANGE = EPOCHS_RANGE[: EPOCHS_RANGE.size // 5]
    LEARNING_RATE_RANGE = LEARNING_RATE_RANGE[: LEARNING_RATE_RANGE.size // 4]
    MOMENTUM_RANGE = MOMENTUM_RANGE[: MOMENTUM_RANGE.size // 4]


## Initial experimentation

The initial experimentation values are set:

In [15]:
INITIAL_NEURONS = NEURONS_RANGE[0]
INITIAL_LAYERS = LAYERS_RANGE[0]
INITIAL_EPOCHS = EPOCHS_RANGE[EPOCHS_RANGE.size // 4]
INITIAL_LEARNING_RATE = LEARNING_RATE_RANGE[LEARNING_RATE_RANGE.size // 2]
INITIAL_MOMENTUM = MOMENTUM_RANGE[MOMENTUM_RANGE.size // 2]

pprint(
    {k: v for k, v in locals().items() if k.isupper() and k.startswith("INITIAL")},
    sort_dicts=False,
)

# This is the real number of experiments that will be ran from the entire total shown above.
real_experiments = NEURONS_RANGE.size * LAYERS_RANGE.size + BEST_FOREACH_PHASE * (
    EPOCHS_RANGE.size + LEARNING_RATE_RANGE.size + MOMENTUM_RANGE.size
)
print(f"Real experiments count: {real_experiments}")


{'INITIAL_NEURONS': 3,
 'INITIAL_LAYERS': 1,
 'INITIAL_EPOCHS': 110,
 'INITIAL_LEARNING_RATE': 0.1039,
 'INITIAL_MOMENTUM': 0.2117}
Real experiments count: 1570


## Hyperparameter tuning phase

Now the hyperparameters are tuned. The first step is to find the best neurons and layers parameters, where
"better" is when the accuracy is higher and the standard deviation is low.

### Neurons and layers tuning

In [30]:
from experimenter.model import ModelParams
import dataclasses

initial_experiment = ModelParams(
    INITIAL_NEURONS,
    layers=INITIAL_LAYERS,
    epochs=INITIAL_EPOCHS,
    learning_rate=INITIAL_LEARNING_RATE,
    momentum=INITIAL_MOMENTUM,
)

print(f'Running {NEURONS_RANGE.size * LAYERS_RANGE.size} experiments')
neurons_result = experimenter.run_all(
    [
        dataclasses.replace(initial_experiment, neurons=neurons, layers=layers)
        for neurons in NEURONS_RANGE
        for layers in LAYERS_RANGE
    ]
)

Running 200 experiments


100%|██████████| 200/200 [02:03<00:00,  1.63it/s]


In [31]:
from typing import List


def to_dataframe(experiments: List[ExperimentResult]):
    return pd.DataFrame(
        [
            {
                **dataclasses.asdict(result.params),
                "result_mean": result.mean,
                "result_std": result.stddev,
            }
            for result in experiments
        ]
    )


def split_best(results):
    results.sort(key=lambda r: r.mean, reverse=True)
    by_mean = results[:BEST_BY_MEAN]

    by_criteria = results[BEST_BY_MEAN:]
    by_criteria.sort(key=experiment_result_criteria, reverse=True)
    by_criteria = by_criteria[:BEST_BY_CUSTOM_CRITERIA]

    return by_mean, by_criteria


def prepare_to_show_best(results, relevant_columns, transform=lambda x: x):
    best = by_mean, by_criteria = split_best(results)
    by_mean_df, by_criteria_df = map(transform, map(to_dataframe, best))

    result_df = transform(to_dataframe(results))

    return (
        by_mean + by_criteria,
        result_df,
        by_mean_df.head(BEST_BY_MEAN)[relevant_columns],
        by_criteria_df.head(BEST_BY_CUSTOM_CRITERIA)[relevant_columns],
    )


(
    best_by_neurons_layers,
    neurons_result_df,
    neurons_by_mean_df,
    neurons_by_criteria_df,
) = prepare_to_show_best(
    neurons_result, ["result_mean", "result_std", "neurons", "layers"]
)

print('Best by mean')
neurons_by_mean_df

Best by mean


,result_mean,result_std,neurons,layers
0,93.103253,1.176310,36,5
1,93.054709,1.082256,33,5
2,92.811873,0.822495,29,4


In [32]:
print('Best by criteria')
neurons_by_criteria_df

Best by criteria


,result_mean,result_std,neurons,layers
0,91.160442,0.338079,35,2
1,92.471830,0.464682,20,2
2,91.597926,0.498681,24,5
3,91.695131,0.559964,31,3
4,91.937850,0.621407,41,2
5,91.209576,0.558163,14,1
6,91.063355,0.566369,8,5
7,91.500484,0.617542,15,3
8,92.131907,0.681950,22,3


In [33]:
import altair as alt


def heatmap(title, source, tooltip="props", **kwargs):
    if tooltip == "props":
        tooltip = source.columns.to_list()

    return (
        alt.Chart(source)
        .mark_rect()
        .encode(tooltip=tooltip, **kwargs)
        .properties(title=title)
        .interactive()
    )


def binify(dataframe, param, bins):
    return [dataframe[dataframe[param].isin(bin)] for bin in bins]


def binified_heatmap(title, source, param, bins, tooltip="props", **kwargs):
    return alt.hconcat(
        *(
            heatmap(title, source[source[param].isin(bin)], tooltip=tooltip, **kwargs)
            for bin in bins
        )
    )


neurons_result_encoding = {"x": "layers:O", "y": "neurons:O", "color": "result_mean:Q"}
neurons_result_tooltip = ["result_mean", "result_std", "neurons", "layers"]
NEURON_BINS = np.array_split(NEURONS_RANGE, NEURONS_RANGE.max() // NEURON_BIN_SIZE + 1)


def binified_neurons_heatmap(title, neurons_df):
    return binified_heatmap(
        title,
        neurons_df,
        param="neurons",
        bins=NEURON_BINS,
        tooltip=neurons_result_tooltip,
        **neurons_result_encoding
    )


binified_neurons_heatmap("Mean precision", neurons_result_df)


alt.HConcatChart(...)

In [34]:
range_neurons_result_df = neurons_result_df[
    neurons_result_df.result_mean >= DETAILED_VIEW_TRESHOLD
]

binified_neurons_heatmap(
    f"Mean precision (>= {DETAILED_VIEW_TRESHOLD})", range_neurons_result_df
)

alt.HConcatChart(...)

### Epochs tuning

In [35]:
print(f"Running {BEST_FOREACH_PHASE * EPOCHS_RANGE.size} experiments")
epochs_results = experimenter.run_all(
    [
        dataclasses.replace(p.params, epochs=epochs)
        for p in best_by_neurons_layers
        for epochs in EPOCHS_RANGE
    ]
)


Running 312 experiments


  5%|▌         | 16/312 [00:21<06:38,  1.35s/it]


KeyboardInterrupt: 

2021-10-26 10:05:00,673	ERROR worker.py:475 -- print_logs: Connection closed by server.
[2021-10-26 10:05:00,679 E 57400 73312] direct_task_transport.cc:556: The worker failed to receive a response from the local raylet. This is most likely because the local raylet has crahsed.
2021-10-26 10:05:00,681	ERROR worker.py:1229 -- listen_error_messages_raylet: Connection closed by server.
2021-10-26 10:05:00,682	ERROR import_thread.py:88 -- ImportThread: Connection closed by server.


In [ ]:
def prepare_reverse_index(source, columns):
    results_reverse_index = {
        tuple(v for k, v in dataclasses.asdict(r.params).items() if k in columns): i
        for i, r in enumerate(source)
    }

    def modify_dataframe(results_df):
        results_df["src_index"] = results_df.apply(
            lambda row: results_reverse_index[tuple(row[k] for k in columns)], axis=1
        )
        return results_df

    return modify_dataframe


(
    best_by_epochs,
    epochs_results_df,
    epochs_by_mean_df,
    epochs_by_criteria_df,
) = prepare_to_show_best(
    epochs_results,
    ["result_mean", "result_std", "neurons", "layers", "epochs", "src_index"],
    transform=prepare_reverse_index(best_by_neurons_layers, ["neurons", "layers"]),
)

print("Best by mean")
epochs_by_mean_df


Best by mean


,result_mean,result_std,neurons,layers,epochs,src_index
0,89.703541,1.027200,12,2,90,0
1,89.460586,1.310644,12,2,70,0
2,89.460114,1.111310,11,2,90,1


In [ ]:
print("Best by criteria")
epochs_by_criteria_df

Best by criteria


,result_mean,result_std,neurons,layers,epochs,src_index
0,89.266293,0.502736,11,2,70,1
1,89.363381,0.748442,11,2,80,1
2,87.906716,0.603003,12,1,90,8
3,88.440696,0.781205,6,1,80,11
4,87.518248,0.695997,12,1,80,8
5,89.072119,0.857919,11,2,60,1
6,88.003685,0.764539,5,2,70,3
7,88.683415,0.854969,6,1,90,11
8,88.683769,0.890495,8,1,90,10


In [ ]:
epochs_results_encoding = {
    "x": "src_index:O",
    "y": "epochs:O",
    "color": "result_mean:Q",
}
epochs_results_tooltip = ["result_mean", "result_std", "neurons", "layers", "epochs", "src_index"]
EPOCHS_BINS = np.array_split(EPOCHS_RANGE, EPOCHS_RANGE.max() // EPOCH_BIN_SIZE + 1)


def binified_epochs_heatmap(title, epochs_df):
    return binified_heatmap(
        title,
        epochs_df,
        "epochs",
        EPOCHS_BINS,
        epochs_results_tooltip,
        **epochs_results_encoding
    )


binified_epochs_heatmap("Mean precision", epochs_results_df)


alt.HConcatChart(...)

In [ ]:
range_epoch_results_df = epochs_results_df[
    epochs_results_df.result_mean >= DETAILED_VIEW_TRESHOLD
]

binified_epochs_heatmap(
    f"Mean precision (>= {DETAILED_VIEW_TRESHOLD})", 
    range_epoch_results_df
)


alt.HConcatChart(...)

### Learning rate tuning

In [ ]:
print(f"Running {BEST_FOREACH_PHASE * LEARNING_RATE_RANGE.size} experiments")

learning_rate_results = experimenter.run_all(
    [
        dataclasses.replace(p.params, learning_rate=learning_rate)
        for p in best_by_epochs
        for learning_rate in LEARNING_RATE_RANGE
    ]
)


Running 84 experiments


  0%|          | 0/84 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
(
    best_by_learning_rate,
    learning_rate_results_df,
    learning_rate_by_mean_df,
    learning_rate_by_criteria_df,
) = prepare_to_show_best(
    learning_rate_results,
    [
        "result_mean",
        "result_std",
        "neurons",
        "layers",
        "epochs",
        "learning_rate",
        "src_index",
    ],
    transform=prepare_reverse_index(best_by_epochs, ["neurons", "layers", "epochs"]),
)

print("Best by mean")
learning_rate_by_mean_df


Best by mean


,result_mean,result_std,neurons,layers,epochs,learning_rate,src_index
0,90.674415,1.161038,11,2,90,0.0422,2
1,90.334727,1.146075,11,2,90,0.0353,2
2,90.141143,1.323766,11,2,80,0.0422,4


In [ ]:
print("Best by criteria")
learning_rate_by_criteria_df

Best by criteria


,result_mean,result_std,neurons,layers,epochs,learning_rate,src_index
0,89.995157,0.561599,12,2,90,0.0422,0
1,89.266293,0.502736,11,2,70,0.0216,3
2,89.849172,0.680307,11,2,80,0.0284,4
3,89.800510,0.709025,12,2,90,0.0353,0
4,89.363381,0.748442,11,2,80,0.0216,4
5,87.906716,0.603003,12,1,90,0.0216,5
6,89.363381,0.779291,11,2,70,0.0284,3
7,89.169088,0.767582,11,2,90,0.0147,2
8,89.120662,0.777560,11,2,80,0.0147,4


In [ ]:
learning_rate_results_encoding = {
    "x": "src_index:O",
    "y": "learning_rate:O",
    "color": "result_mean:Q",
}
learning_rate_results_tooltip = [
    "result_mean",
    "result_std",
    "neurons",
    "layers",
    "epochs",
    "learning_rate",
    "src_index",
]
LEARNING_RATE_BINS = np.array_split(
    LEARNING_RATE_RANGE, LEARNING_RATE_RANGE.max() // LEARNING_RATE_BIN_SIZE + 1
)


def binified_learning_rate_heatmap(title, lr_df):
    return binified_heatmap(
        title,
        lr_df,
        "learning_rate",
        LEARNING_RATE_BINS,
        learning_rate_results_tooltip,
        **learning_rate_results_encoding
    )


binified_learning_rate_heatmap("Mean precision", learning_rate_results_df)


alt.HConcatChart(...)

In [ ]:
range_learning_rate_results_df = learning_rate_results_df[
    learning_rate_results_df.result_mean >= DETAILED_VIEW_TRESHOLD
]

binified_learning_rate_heatmap(
    f"Mean precision (>= {DETAILED_VIEW_TRESHOLD})", range_learning_rate_results_df
)


alt.HConcatChart(...)

### Momentum tuning

In [ ]:
print(f"Running {BEST_FOREACH_PHASE * MOMENTUM_RANGE.size} experiments")

momentum_results = experimenter.run_all(
    [
        dataclasses.replace(p.params, momentum=momentum)
        for p in best_by_learning_rate
        for momentum in MOMENTUM_RANGE
    ]
)


Running 84 experiments


100%|██████████| 84/84 [00:29<00:00,  2.80it/s]


## Best results

The best results found are show below.

In [ ]:
(
    best_by_momentum,
    momentum_results_df,
    momentum_by_mean_df,
    momentum_by_criteria_df,
) = prepare_to_show_best(
    momentum_results,
    [
        "result_mean",
        "result_std",
        "neurons",
        "layers",
        "epochs",
        "learning_rate",
        "momentum",
        "src_index",
    ],
    transform=prepare_reverse_index(
        best_by_learning_rate, ["neurons", "layers", "epochs", "learning_rate"]
    ),
)

print("Best by mean")
momentum_by_mean_df


Best by mean


,result_mean,result_std,neurons,layers,epochs,learning_rate,momentum,src_index
0,90.820046,1.159659,11,2,90,0.0422,0.0907,0
1,90.771502,1.205281,11,2,90,0.0422,0.0772,0
2,90.722958,1.159224,11,2,90,0.0422,0.0638,0


In [ ]:
print("Best by criteria")
momentum_by_criteria_df

Best by criteria


,result_mean,result_std,neurons,layers,epochs,learning_rate,momentum,src_index
0,89.898188,0.519396,12,2,90,0.0422,0.0907,3
1,89.995157,0.561599,12,2,90,0.0422,0.0503,3
2,89.995039,0.563733,12,2,90,0.0422,0.0234,3
3,89.946496,0.567473,12,2,90,0.0422,0.0369,3
4,89.266293,0.502736,11,2,70,0.0216,0.0503,4
5,89.994921,0.606169,12,2,90,0.0422,0.0100,3
6,89.946141,0.632913,12,2,90,0.0353,0.0907,6
7,89.946614,0.643347,12,2,90,0.0422,0.0638,3
8,89.314837,0.581478,11,2,70,0.0216,0.0638,4


In [ ]:
momentum_results_encoding = {
    "x": "src_index:O",
    "y": "momentum:O",
    "color": "result_mean:Q",
}
momentum_results_tooltip = [
    "result_mean",
    "result_std",
    "neurons",
    "layers",
    "epochs",
    "learning_rate",
    "momentum",
    "src_index",
]
MOMENTUM_BINS = np.array_split(
    MOMENTUM_RANGE, MOMENTUM_RANGE.max() // MOMENTUM_BIN_SIZE + 1
)


def binified_momentum_heatmap(title, momentum_df):
    return binified_heatmap(
        title,
        momentum_df,
        "momentum",
        MOMENTUM_BINS,
        momentum_results_tooltip,
        **momentum_results_encoding
    )


binified_momentum_heatmap("Mean precision", momentum_results_df)


alt.HConcatChart(...)

In [ ]:
range_momentum_results_df = momentum_results_df[
    momentum_results_df.result_mean >= DETAILED_VIEW_TRESHOLD
]

binified_momentum_heatmap(
    f"Mean precision (>= {DETAILED_VIEW_TRESHOLD})", range_momentum_results_df
)


alt.HConcatChart(...)